In [2]:
import os
from modulus.sym.hydra import to_absolute_path
from csv_rw import csv_to_dict
import csv
import numpy as np
import glob
from multiprocessing import Process
from fwdFacingStep import ffs, param_ranges, Re, Ho, Lo


In [3]:
def readFile(file_path):
    with open(file_path, "r") as datafile:
        data = []
        reader = csv.reader(datafile, delimiter=",")
        for row in reader:
            columns = [row[1]]
            data.append(columns)
        last_row = float(data[-1][0])
        return last_row

In [41]:
ansysFilePath="./ansysDPsValidation.csv"
resultsFilePath="./results.csv"
outputsPath="./outputs/fwdFacingStep"

models = ["old_dataPlusPhysics3600@300k", "data3600PlusPhysicsLambda05@300k", "data3600PlusPhysicsLambda1@300k", "old_physicsOnly@500k", "physicsOnly@500k"]

diffSumSkip = ["DP 5","DP 36","DP 79","DP 86"]


with open(ansysFilePath, "r") as ansysFile,  open(resultsFilePath, "w") as resultsFile:
        reader = csv.reader(ansysFile, delimiter=",")
        writer = csv.writer(resultsFile, delimiter=",")
        diffDict= {}
        for model in models:
                diffDict[model] = 0
                # print(type(diffDict[model]))
        for i, row in enumerate(reader):
                if i == 0:
                        # add columns for difference modulus vs ansys
                        row.insert(5, "Diff Delta Cp [%]")
                        # row.insert(7, "Diff Delta Cptot [%]")
                        writer.writerow(row)
                else:
                        # add ansys results to table
                        row.insert(5, "-")
                        # row.insert(7, "-")
                        writer.writerow(row)
                        for model in models:
                                monitorFilePath = os.path.join(outputsPath, model, "monitors")
                                # read pressure monitors
                                filePattern="downstreamPressure_" + row[0].replace(" ","") + "_*"
                                monitorFile=glob.glob(os.path.join(monitorFilePath, filePattern))
                                DSP=readFile(monitorFile[0])
                                # filePattern="downstreamPressureTot_" + row[0].replace(" ","") + "_*"
                                # monitorFile=glob.glob(os.path.join(monitorFilePath, filePattern))
                                # DSPtot=readFile(monitorFile[0])
                                filePattern="upstreamPressure_" + row[0].replace(" ","") + "_*"
                                monitorFile=glob.glob(os.path.join(monitorFilePath, filePattern))
                                USP=readFile(monitorFile[0])
                                # filePattern="upstreamPressureTot_" + row[0].replace(" ","") + "_*"
                                # monitorFile=glob.glob(os.path.join(monitorFilePath, filePattern))
                                # USPtot=readFile(monitorFile[0])
                                
                                # calculate modulus pressure drop and difference modulus vs ansys
                                deltaCp=2*(USP-DSP)
                                # deltaCptot=2*(USPtot-DSPtot)
                                diffDeltaCp=100*(deltaCp-float(row[4]))/float(row[4])
                                # diffDeltaCptot=100*(deltaCptot-float(row[6]))/float(row[6])
                                
                                print(row[0])
                                if row[0] not in diffSumSkip:
                                        print("ns", row[0])
                                        diffDict[model] += abs(diffDeltaCp)
                                
                                # add modulus results to table
                                modRow=[row[0] + " " + model] + row[1:4] + [deltaCp] + [diffDeltaCp] + [DSP] + [USP]
                                # modRow=[row[0] + " " + model] + row[1:4] + [deltaCp] + [diffDeltaCp] + [deltaCptot] + [diffDeltaCptot] + [DSP] + [DSPtot] + [USP] + [USPtot]
                                writer.writerow(modRow)
        # Add deviation sum
        for model in models:
                deviationRow = ["Sum " + model] + ["-"] + ["-"] + ["-"] + ["-"] + [diffDict[model]] + ["-"] + ["-"]
                writer.writerow(deviationRow)
                
                                
                                


DP 5
DP 5
DP 5
DP 5
DP 5
DP 36
DP 36
DP 36
DP 36
DP 36
DP 79
DP 79
DP 79
DP 79
DP 79
DP 86
DP 86
DP 86
DP 86
DP 86
DP 154
ns DP 154
DP 154
ns DP 154
DP 154
ns DP 154
DP 154
ns DP 154
DP 154
ns DP 154
DP 155
ns DP 155
DP 155
ns DP 155
DP 155
ns DP 155
DP 155
ns DP 155
DP 155
ns DP 155
DP 156
ns DP 156
DP 156
ns DP 156
DP 156
ns DP 156
DP 156
ns DP 156
DP 156
ns DP 156
DP 157
ns DP 157
DP 157
ns DP 157
DP 157
ns DP 157
DP 157
ns DP 157
DP 157
ns DP 157
DP 158
ns DP 158
DP 158
ns DP 158
DP 158
ns DP 158
DP 158
ns DP 158
DP 158
ns DP 158
DP 159
ns DP 159
DP 159
ns DP 159
DP 159
ns DP 159
DP 159
ns DP 159
DP 159
ns DP 159
DP 160
ns DP 160
DP 160
ns DP 160
DP 160
ns DP 160
DP 160
ns DP 160
DP 160
ns DP 160
DP 161
ns DP 161
DP 161
ns DP 161
DP 161
ns DP 161
DP 161
ns DP 161
DP 161
ns DP 161
DP 162
ns DP 162
DP 162
ns DP 162
DP 162
ns DP 162
DP 162
ns DP 162
DP 162
ns DP 162
DP 163
ns DP 163
DP 163
ns DP 163
DP 163
ns DP 163
DP 163
ns DP 163
DP 163
ns DP 163
DP 164
ns DP 164
DP 164
ns DP 164
D

In [42]:
for re in range(100,1100, 100):
    F = np.load("optResultsF" + str(re) + ".npy")
    X = np.load("optResultsX" + str(re) + ".npy")
    popF = np.load("popF" + str(re) + ".npy")
    popX = np.load("popX" + str(re) + ".npy") 
    # print("Re " + str(re) + " F: ", F)
    print("Re " + str(re) + " X: ", X)

Re 100 X:  [0.70045392 0.32997775]
Re 200 X:  [0.45144677 0.30338055]
Re 300 X:  [0.44996036 0.30353034]
Re 400 X:  [0.20000089 0.32999946]
Re 500 X:  [0.45093736 0.30275938]
Re 600 X:  [0.45125546 0.30468607]
Re 700 X:  [0.2000014  0.32999907]
Re 800 X:  [0.69735089 0.32999746]
Re 900 X:  [0.69835677 0.32999852]


FileNotFoundError: [Errno 2] No such file or directory: 'optResultsF1000.npy'